In [1]:
import pandas as pd
import pickle

In [2]:
data = pd.read_pickle(r'C:\Users\PIYUSH\Downloads\merged_training.pkl')

In [3]:
data.head()

,text,emotions
27383,i feel awful about it too because it s my job ...,sadness
110083,im alone i feel awful,sadness
140764,ive probably mentioned this before but i reall...,joy
100071,i was feeling a little low few days back,sadness
2837,i beleive that i am much more sensitive to oth...,love


In [4]:
data=data.iloc[:80000]

In [5]:
data.shape

(80000, 2)

In [6]:
data.columns = ['content', 'sentiments']

In [7]:
data['sentiments'].unique()

array(['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'],
      dtype=object)

In [8]:
import re
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [9]:
#Making all letters lowercase
data['content'] = data['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation, Symbols
data['content'] = data['content'].str.replace('[^\w\s]',' ')
#Removing Stop Words using NLTK
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Lemmatisation
from textblob import Word
data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'] = data['content'].apply(lambda x: " ".join(de_repeat(x) for x in x.split()))


In [10]:
# # Code to find the top 10,000 rarest words appearing in the data
# freq = pd.Series(' '.join(data['content']).split()).value_counts()[-10000:]
# # Removing all those rarely appearing words from the data
# freq = list(freq.index)
# data['content'] = data['content'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [12]:
data.head()

,content,sentiments
27383,feel awful job get position succeed happen,sadness
110083,im alone feel awful,sadness
140764,ive probably mentioned really feel proud actua...,joy
100071,feeling little low day back,sadness
2837,beleive much sensitive people feeling tend com...,love


In [59]:
l=0
for i in range(data.shape[0]):
    l=max(len(data.iloc[i]['content'].split(' ')),l)

In [60]:
l

45

enthusiasm=happy=fun
empty=neutral
boredom=sadness

In [13]:
data.head()

,content,sentiments
27383,feel awful job get position succeed happen,sadness
110083,im alone feel awful,sadness
140764,ive probably mentioned really feel proud actua...,joy
100071,feeling little low day back,sadness
2837,beleive much sensitive people feeling tend com...,love


In [14]:
#Encoding output labels 'sadness' as '1' & 'happiness' as '0'
from sklearn import preprocessing
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.sentiments.values)
# Splitting into training and testing data in 90:10 ratio

In [15]:
y[]

array([4, 4, 2, ..., 2, 4, 4], dtype=int64)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data.content.values, y, test_size=0.1, shuffle=True)


In [19]:
y_val.shape

(8000,)

In [66]:
# Extracting TF-IDF parameters
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, analyzer='word',ngram_range=(1,3))
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)

In [67]:
X_train_tfidf=X_train_tfidf.toarray()

In [68]:
X_val_tfidf=X_val_tfidf.toarray()

In [69]:
X_val_tfidf=X_val_tfidf.reshape(X_val_tfidf.shape[0],1,X_val_tfidf.shape[1])

In [70]:
X_train_tfidf=X_train_tfidf.reshape(X_train_tfidf.shape[0],1,X_train_tfidf.shape[1])

In [71]:
from keras.utils import to_categorical
y_label=to_categorical(y_train)
val_label=to_categorical(y_val)
y_label.shape

(72000, 6)

In [72]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential

In [73]:
classifier = Sequential()
classifier.add(LSTM(256,input_shape=(1,1000),return_sequences=True))
classifier.add(Dropout(0.5))
classifier.add(LSTM(256))
classifier.add(Dropout(0.5))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))
classifier.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 1, 256)            1287168   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 256)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 1542      
_________________________________________________________________
activation_4 (Activation)    (None, 6)                 0         
Total params: 1,814,022
Trainable params: 1,814,022
Non-trainable params: 0
____________________________________________

In [74]:
val_list=[X_val_tfidf,val_label]

In [75]:
classifier.compile( loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'] )

In [81]:
classifier.fit(X_train_tfidf,y_label,batch_size=120,epochs=20,validation_split=0.1)

Train on 64800 samples, validate on 7200 samples
Epoch 1/20
64800/64800 [==============================] - 22s 342us/sample - loss: 0.2656 - accuracy: 0.8904 - val_loss: 0.2481 - val_accuracy: 0.8957
Epoch 2/20
64800/64800 [==============================] - 21s 324us/sample - loss: 0.2598 - accuracy: 0.8910 - val_loss: 0.2587 - val_accuracy: 0.8900
Epoch 3/20
64800/64800 [==============================] - 20s 316us/sample - loss: 0.2521 - accuracy: 0.8953 - val_loss: 0.2688 - val_accuracy: 0.8854
Epoch 4/20
64800/64800 [==============================] - 20s 316us/sample - loss: 0.2485 - accuracy: 0.8959 - val_loss: 0.2738 - val_accuracy: 0.8858
Epoch 5/20
64800/64800 [==============================] - 22s 332us/sample - loss: 0.2402 - accuracy: 0.8995 - val_loss: 0.2786 - val_accuracy: 0.8839
Epoch 6/20
64800/64800 [==============================] - 21s 324us/sample - loss: 0.2371 - accuracy: 0.9004 - val_loss: 0.2834 - val_accuracy: 0.8839
Epoch 7/20
64800/64800 [=====================

In [83]:
y_prob = classifier.predict(X_val_tfidf) 
y_classes = y_prob.argmax(axis=-1)

In [84]:
y_classes

array([2, 2, 3, ..., 2, 1, 5], dtype=int64)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_classes)

0.852

In [95]:
import numpy as np
embeddings = {}
with open(r'G:\data\glove.6B.100d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        #print(word)
        #print(coeffs)
        embeddings[word] = coeffs

In [97]:
embeddings['the'].shape

(100,)

In [101]:
def getOutputEmbeddings(X):
    embedding_matrix_output = np.zeros((X.shape[0],l,100))
    for ix in range(X.shape[0]):
        sent = X['content'].iloc[ix].split()
        for jx in range(len(sent)):
            try:
                embedding_matrix_output[ix][jx] = embeddings[sent[jx].lower()]
            except:
                continue
            
    return embedding_matrix_output

In [102]:
data['content'].iloc[0]

'feel awful job get position succeed happen'

In [106]:
glove_data=getOutputEmbeddings(data)

In [107]:
glove_data.shape

(80000, 45, 100)

In [109]:
y.shape

(80000,)

In [110]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(glove_data, y, test_size=0.1, shuffle=True)

In [112]:
from keras.utils import to_categorical
y_label=to_categorical(y_train)
val_label=to_categorical(y_val)
y_label.shape

(72000, 6)

In [113]:
X_train.shape

(72000, 45, 100)

In [114]:
classifier = Sequential()
classifier.add(LSTM(128,input_shape=(45,100),return_sequences=True))
classifier.add(Dropout(0.5))
classifier.add(LSTM(128))
classifier.add(Dropout(0.5))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))
classifier.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 45, 128)           117248    
_________________________________________________________________
dropout_10 (Dropout)         (None, 45, 128)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 774       
_________________________________________________________________
activation_5 (Activation)    (None, 6)                 0         
Total params: 249,606
Trainable params: 249,606
Non-trainable params: 0
________________________________________________

In [115]:
classifier.compile( loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'] )

In [116]:
classifier.fit(X_train,y_label,batch_size=120,epochs=20,validation_split=0.1)

Train on 64800 samples, validate on 7200 samples
Epoch 1/20
64800/64800 [==============================] - 182s 3ms/sample - loss: 1.1351 - accuracy: 0.5701 - val_loss: 0.7749 - val_accuracy: 0.7147
Epoch 2/20
64800/64800 [==============================] - 173s 3ms/sample - loss: 0.5057 - accuracy: 0.8183 - val_loss: 0.3647 - val_accuracy: 0.8792
Epoch 3/20
64800/64800 [==============================] - 175s 3ms/sample - loss: 0.2675 - accuracy: 0.9018 - val_loss: 0.2366 - val_accuracy: 0.9101
Epoch 4/20
64800/64800 [==============================] - 177s 3ms/sample - loss: 0.1977 - accuracy: 0.9187 - val_loss: 0.1946 - val_accuracy: 0.9167
Epoch 5/20
64800/64800 [==============================] - 182s 3ms/sample - loss: 0.1648 - accuracy: 0.9257 - val_loss: 0.1757 - val_accuracy: 0.9242
Epoch 6/20
64800/64800 [==============================] - 176s 3ms/sample - loss: 0.1446 - accuracy: 0.9302 - val_loss: 0.1690 - val_accuracy: 0.9257
Epoch 7/20
64800/64800 [===========================

In [117]:
y_prob = classifier.predict(X_val) 
y_classes = y_prob.argmax(axis=-1)

In [118]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_classes)

0.928875

In [120]:
def testing(X):
    embedding_matrix_output = np.zeros((1,l,100))
    sent = X.split()
    for jx in range(len(sent)):
            try:
                embedding_matrix_output[0][jx] = embeddings[sent[jx].lower()]
            except:
                continue
            
    return embedding_matrix_output

In [132]:
arr=testing('whatever happened it was very bad')
arr.shape

(1, 45, 100)

In [133]:
y_prob = classifier.predict(arr) 
y_classes = y_prob.argmax(axis=-1)

In [134]:
y_classes

array([4], dtype=int64)